In [1]:
import torch
import torch.nn as nn
import torch.autograd
from torch.autograd import Variable
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.utils.data as data

In [2]:
#### cvt2tensor = transforms.ToTensor()
cvt2pil = transforms.ToPILImage()

class Spacenet_Dataset(data.Dataset):
    def __init__(self, annotationsFile,train = True, transform=None, target_transform = None):
        lines = open(annotationsFile,'r').readlines()
        self.transform=transform
        self.target_transform=target_transform
        self.data=[]
        for line in lines:
            ip,out = line.split(' ')
            out = out[:-5]+'segobj.png'
            item = [ip,out]
            self.data.append(item)
        inp_img_paths = self.data[:][0]
        out_img_paths = self.data[:][1]
              
    def getImage(self,path):
        pil_image = Image.open(path)
        pil_image = pil_image.convert('RGB')
        return pil_image

    def __getitem__(self, index):
        cur_data = self.data[index]
        ip_path= cur_data[0]
        out_path = cur_data[1]
        in_image = self.getImage(ip_path)
        out_image = self.getImage(out_path)
        
        if self.transform is not None:
            in_image = self.transform(in_image)
            out_image = self.transform(out_image)
        return in_image, out_image

imgTransform = transforms.Compose([transforms.Scale((400, 400)),
                                   transforms.ToTensor()])

trainData = Spacenet_Dataset("/home/dipshil/Spacenet_things/annotations/trainval.txt",transform=imgTransform)
valData = Spacenet_Dataset("/home/dipshil/Spacenet_things/annotations/test.txt",transform=imgTransform)
inp,out = trainData[200]

In [3]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        # Convolutional layers.
        self.conv1 = nn.Conv2d(3, 64, 8, stride = 2)
        self.conv2 = nn.Conv2d(64, 128, 6, stride = 2)
        self.conv3 = nn.Conv2d(128, 256, 7,stride = 3)
        self.conv4 = nn.Conv2d(256, 512, 3,stride = 2, padding =2)
        
        self.deconv1 = nn.ConvTranspose2d(512,256,3,stride=2,padding=1)
        self.deconv2 = nn.ConvTranspose2d(256,128,7,stride=3)
        self.deconv3 = nn.ConvTranspose2d(128,64,6,stride=2)
        self.deconv4 = nn.ConvTranspose2d(64,3,8,stride=2,padding=1)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.relu(self.deconv1(out))
        out = F.relu(self.deconv2(out))
        out = F.relu(self.deconv3(out))
        out = F.relu(self.deconv4(out))
        
        return out

In [4]:
import random

def get_sample():
    inp,out = trainData[random.randint(0,3673)] # change it to Valdata if necessary, dont forget to change value inside rand()
    inp = inp.view(-1,3,400,400)
    model = MyNet().cuda()
    sample_ip = torch.autograd.Variable(inp).cuda()
    sample_out = model(sample_ip)
    sample_out = sample_out.view(-1,400,400)
    sample_out = cvt2pil(sample_out.data.cpu())
    sample_inp = cvt2pil(inp.view(-1,400,400)) 
    plt.imshow(sample_inp)
    plt.show()
    plt.imshow(sample_out)
    plt.show()

In [5]:
from tqdm import tqdm as tqdm

train_loss=[]
test_loss=[]

def train_rnn_model(model, criterion, optimizer, trainData, valData, n_epochs = 10, use_gpu = False):      
    
    if use_gpu:
        print('using GPU!')
        model = model.cuda()
        criterion = criterion.cuda()
        
    # Training loop.
    for epoch in range(0, n_epochs):
        cum_loss = 0.0
        accuracy = 0
        
        # Make a pass over the training data.
        t = tqdm(trainData, desc = 'Training epoch %d' % epoch)
        model.train()  # This is important to call before training!
        for (ip_image, out_image) in enumerate(t): 

            input_image = torch.autograd.Variable(ip_image)
            target = torch.autograd.Variable(out_image)

            if use_gpu:
                input_image = input_image.cuda()
                target = target.cuda()
                
            y_ = model(input_image)

            loss = criterion(y_, target)
            cum_loss += loss.data[0]

            optimizer.zero_grad()

            loss.backward()
                    # Weight and bias updates.
            optimizer.step()                

            # logging information.
            t.set_postfix(loss = cum_loss / (1 + i)) 

        train_loss.append(cum_loss/(i+1))


        # Make a pass over the validation data.
        cum_loss = 0.0
        accuracy = 0
        
        t = tqdm(valData, desc = 'Validation epoch %d' % epoch)
        model.eval()  # This is important to call before evaluating!
        for (ip_image,out_image) in enumerate(t): 
        
            input_image = torch.autograd.Variable(ip_image)
            target = torch.autograd.Variable(out_image)

            if use_gpu:
                input_image = input_image.cuda()
                target = target.cuda()
                
            y_ = model(input_image)

            loss = criterion(y_, target)
            cum_loss += loss.data[0]
            
        test_loss.append(cum_loss/(i+1))
    return model

model = MyNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

# Train the previously defined model.
trained_model = train_rnn_model(model, criterion, optimizer, trainData, valData, n_epochs = 10, use_gpu = False)

NotImplementedError: 